In [1]:
##Lets webscrap Toronto housing! 

In [78]:
from bs4 import BeautifulSoup # For HTML parsing
import requests # Website connections
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
import pandas as pd # For converting results to a dataframe and bar chart plots
import json # For parsing json
from requests import get
from random import randint
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np,re

In [79]:
#get the first page of the Toronto housing prices
response = get('https://toronto.craigslist.org/search/apa?hasPic=1&availabilityMode=0&sale_date=all+dates')
#delete  post of an housing option without a pic using their filter

In [80]:
print(response.text[:500])

﻿<!DOCTYPE html>
<html class="no-js"><head>
    <title>toronto apts/housing for rent  - craigslist</title>

    <meta name="description" content="toronto apts/housing for rent  - craigslist">
    <meta http-equiv="X-UA-Compatible" content="IE=Edge"/>
    <link rel="canonical" href="https://toronto.craigslist.org/search/apa">
    <link rel="alternate" type="application/rss+xml" href="https://toronto.craigslist.org/search/apa?availabilityMode=0&amp;format=rss&amp;hasPic=1" title="RSS feed for crai


In [81]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [82]:
type(html_soup)

bs4.BeautifulSoup

In [83]:
posts = html_soup.find_all('li', class_= 'result-row')
print(type(posts)) #to double check that I got a ResultSet
print(len(posts)) #to double check I got 50 (elements/page)

<class 'bs4.element.ResultSet'>
120


In [84]:
#The first post on craiglist
post_one = posts[0]
post_one

<li class="result-row" data-pid="7049400123" data-repost-of="7032415325">
<a class="result-image gallery" data-ids="1:00G0G_j359mVoVuHf,1:00i0i_MdiFI6G4FK,1:00d0d_ijGFObVDDKg,1:00101_miK9H6FJWB,1:00Q0Q_jycbaLjiSgP,1:00I0I_8hYtyNzRzU3,1:00x0x_17SFLEBBuVo,1:01515_5cQF1B0oclh,1:00101_jOFAdmDEZir,1:00R0R_55MO1yXKJl5,1:00q0q_5nRYGvidZ0A,1:01414_2xJFyK6wdul" href="https://toronto.craigslist.org/tor/apa/d/toronto-stunning-1-bedroom-condo-with/7049400123.html">
<span class="result-price">$3495</span>
</a>
<p class="result-info">
<span class="icon icon-star" role="button">
<span class="screen-reader-text">favorite this post</span>
</span>
<time class="result-date" datetime="2020-01-11 15:57" title="Sat 11 Jan 03:57:42 PM">Jan 11</time>
<a class="result-title hdrlnk" data-id="7049400123" href="https://toronto.craigslist.org/tor/apa/d/toronto-stunning-1-bedroom-condo-with/7049400123.html">Stunning 1 Bedroom Condo with Granite Counters and A/C</a>
<span class="result-meta">
<span class="result-pri

In [85]:
#grab the price of the first post
post_one_price = post_one.a.text
post_one_price

'\n$3495\n'

In [86]:
#grab the time of the post in datetime format to save on cleaning efforts
post_one_time = post_one.find('time', class_= 'result-date')
post_one_datetime = post_one_time['datetime']

#split them and grab each element of the resulting list to get date and time
post_one_datetime = post_one_datetime.split(' ')
post_one_date = post_one_datetime[0]
post_one_date

#element 2 is the time
post_one_time = post_one_datetime[1]

post_one_time

'15:57'

In [87]:
#title is a and that class, link is grabbing the href attribute of that variable
post_one_title = post_one.find('a', class_='result-title hdrlnk')
post_one_link = post_one_title['href']
post_one_link

'https://toronto.craigslist.org/tor/apa/d/toronto-stunning-1-bedroom-condo-with/7049400123.html'

In [88]:
#easy to grab the post title by taking the text element of the title variable
post_one_title_text = post_one_title.text
post_one_title_text

'Stunning 1 Bedroom Condo with Granite Counters and A/C'

In [89]:
#the neighborhood is grabbed by finding the span class 'result-hood' and pulling the text element from that
post_one_hood = post_one.find('span', class_= 'result-hood').text
post_one_hood

' (YORKVILLE)'

In [90]:
#getting sqft
post_one_sqft = post_one.find('span', class_ = 'housing').text.split()[2][:-3]
post_one_sqft


'650'

In [91]:
class Site:
    def __init__(self):
        self.selected_date=['2019-12-01','2020-01-11']
        self.current_response =None
        self.current_html_soup=None
        self.headers = {
            'User-Agent': 'My User Agent 1.0',
            'From': 'youremail@domain.com'  # This is another valid field
        }


    def get_and_make_soup(self,date):
        self.current_response=get('https://toronto.craigslist.org/search/apa?hasPic=1&availabilityMode=0&sale_date='+date,headers=self.headers)
        self.current_html_soup = BeautifulSoup(self.current_response.text, 'html.parser')


    def getting_pages_post(self):

        results_num = self.current_html_soup.find('div', class_='search-legend')
        results_total = int(results_num.find('span', class_='totalcount').text)
        pages = np.arange(0, results_total, 120)

        iterations = 0

        post_timing = []
        post_hoods = []
        post_title_texts = []
        post_links = []
        post_prices = []
        sqfts = []
        bedroom_counts = []
        for page in pages:

            # get request
            response = get("https://toronto.craigslist.org/search/eby/apa?"
                   + "s="
                   + str(page)
                   + "&hasPic=1"
                   + "&availabilityMode=0")

            sleep(randint(1, 5))

            # throw warning for status codes that are not 200
            if response.status_code != 200:
                warn('Request: {}; Status code: {}'.format(requests, response.status_code))

            # define the html text
            page_html = BeautifulSoup(response.text, 'html.parser')

            # define the posts
            posts = page_html.find_all('li', class_='result-row')

            # extract data item-wise
            for post in posts:

                if post.find('span', class_='result-hood') is not None:

                    # posting date
                    # grab the datetime element 0 for date and 1 for time
                    post_datetime = post.find('time', class_='result-date')['datetime']
                    post_timing.append(post_datetime)

                    # neighborhoods
                    post_hood = post.find('span', class_='result-hood').text
                    post_hoods.append(post_hood)

                    # title text
                    post_title = post.find('a', class_='result-title hdrlnk')
                    post_title_text = post_title.text
                    post_title_texts.append(post_title_text)

                    # post link
                    post_link = post_title['href']
                    post_links.append(post_link)

                    if post.find('span', class_='result-price') is None:
                        post_price = np.nan
                    else:
                        post_price = post.a.text.strip().replace("\n$", "")
                        post_prices.append(post_price)
                    if post.find('span', class_='housing') is not None:

                        # if the first element is accidentally square footage
                        if 'ft2' in post.find('span', class_='housing').text.split()[0]:

                        # make bedroom nan
                            bedroom_count = np.nan
                            bedroom_counts.append(bedroom_count)

                            # make sqft the first element
                            sqft = int(post.find('span', class_='housing').text.split()[0][:-3])
                            sqfts.append(sqft)

                            # if the length of the housing details element is more than 2
                        elif len(post.find('span', class_='housing').text.split()) > 2:

                            # therefore element 0 will be bedroom count
                            bedroom_count = post.find('span', class_='housing').text.replace("br", "").split()[0]
                            bedroom_counts.append(bedroom_count)

                            # and sqft will be number 3, so set these here and append
                            sqft = int(post.find('span', class_='housing').text.split()[2][:-3])
                            sqfts.append(sqft)

                        # if there is num bedrooms but no sqft
                        elif len(post.find('span', class_='housing').text.split()) == 2:

                            # therefore element 0 will be bedroom count
                            bedroom_count = post.find('span', class_='housing').text.replace("br", "").split()[0]
                            bedroom_counts.append(bedroom_count)

                            # and sqft will be number 3, so set these here and append
                            sqft = np.nan
                            sqfts.append(sqft)

                        else:
                            bedroom_count = np.nan
                            bedroom_counts.append(bedroom_count)

                            sqft = np.nan
                            sqfts.append(sqft)

                    # if none of those conditions catch, make bedroom nan, this won't be needed
                    else:
                        bedroom_count = np.nan
                        bedroom_counts.append(bedroom_count)

                        sqft = np.nan
                        sqfts.append(sqft)
                        #    bedroom_counts.append(bedroom_count)

                        #    sqft = np.nan
                        #    sqfts.append(sqft)

            iterations += 1
            print("Finished iteration: " + str(iterations))
        return post_timing,post_hoods,post_title_text,post_links,post_prices,sqfts,bedroom_counts

    def fetching_data(self):
        post_timing,post_hoods,post_title_texts,post_links,post_prices,sqfts,bedroom_counts=self.getting_pages_post()

        to_rooms = pd.DataFrame({'Date_Posted': post_timing,
                       'Neighborhood': post_hoods,
                       'Post_Title': post_title_texts,
                       'URL': post_links,
                       'Price': post_prices,
                       'Sqft':sqfts,
                        'Bedrooms':bedroom_counts})


        to_rooms['Price']=to_rooms['Price'].str.replace('$','')
        to_rooms['Neighborhood']=to_rooms['Neighborhood'].str.replace('(','')
        to_rooms['Neighborhood']=to_rooms['Neighborhood'].str.replace(')','')
        to_rooms['Bedrooms'] = to_rooms['Bedrooms'].apply(lambda x: float(x))
        #to_rooms.info()
        to_rooms['Date_Posted'] = pd.to_datetime(to_rooms['Date_Posted'])

        to_rooms['Price'] = to_rooms['Price'].astype(str)

        to_rooms['Price'] = to_rooms['Price'].apply(lambda x: re.sub(r'[\n$]+', '', x))

        to_rooms['Price'].replace('', np.nan, inplace=True)
        to_rooms = to_rooms.dropna()
        to_rooms['Price'] = to_rooms['Price'].astype(int)
        #to_rooms.describe()
        #to_rooms.head(200)
        to_rooms = to_rooms.drop_duplicates(subset='Post_Title')
        to_rooms.head(100)

    def run(self):
        for each in self.selected_date:
            ## perform single iteration using break statement
            ## if more iterations, remove break statements..
            self.get_and_make_soup(each)
            self.fetching_data()
            break
            

s=Site()
s.run()


Finished iteration: 1


In [59]:
import pandas as pd

to_rooms = pd.DataFrame({'Date_Posted': post_timing,
                       'Neighborhood': post_hoods,
                       'Post_Title': post_title_texts,
                       'URL': post_links,
                       'Price': post_prices,
                       'Sqft':sqfts,
                        'Bedrooms':bedroom_counts})

In [60]:
to_rooms['Post_Title'].unique().shape

(83,)

In [61]:
to_rooms['Price']=to_rooms['Price'].str.replace('$','')
to_rooms['Neighborhood']=to_rooms['Neighborhood'].str.replace('(','')
to_rooms['Neighborhood']=to_rooms['Neighborhood'].str.replace(')','')
to_rooms['Bedrooms'] = to_rooms['Bedrooms'].apply(lambda x: float(x))
print(to_rooms.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 7 columns):
Date_Posted     2075 non-null object
Neighborhood    2075 non-null object
Post_Title      2075 non-null object
URL             2075 non-null object
Price           2075 non-null object
Sqft            1100 non-null float64
Bedrooms        2000 non-null float64
dtypes: float64(2), object(5)
memory usage: 113.6+ KB
None


In [62]:
to_rooms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 7 columns):
Date_Posted     2075 non-null object
Neighborhood    2075 non-null object
Post_Title      2075 non-null object
URL             2075 non-null object
Price           2075 non-null object
Sqft            1100 non-null float64
Bedrooms        2000 non-null float64
dtypes: float64(2), object(5)
memory usage: 113.6+ KB


In [63]:
to_rooms['Date_Posted'] = pd.to_datetime(to_rooms['Date_Posted'])

In [64]:
to_rooms['Price'] = to_rooms['Price'].astype(str)

In [65]:
import re

to_rooms['Price'] = to_rooms['Price'].apply(lambda x: re.sub(r'[\n$]+', '', x))

#replace '' with NaN values in the df before turning the column to int so data shape is maintained

In [66]:
import numpy as np
to_rooms['Price'].replace('', np.nan, inplace=True)

In [67]:
#drop the NAs because we dont like na
to_rooms = to_rooms.dropna()

In [68]:
to_rooms.info() #we still need to change the price to an int

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1075 entries, 0 to 2073
Data columns (total 7 columns):
Date_Posted     1075 non-null datetime64[ns]
Neighborhood    1075 non-null object
Post_Title      1075 non-null object
URL             1075 non-null object
Price           1075 non-null object
Sqft            1075 non-null float64
Bedrooms        1075 non-null float64
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 67.2+ KB


In [69]:
#now turn the price to int as well
to_rooms['Price'] = to_rooms['Price'].astype(int)

In [70]:
to_rooms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1075 entries, 0 to 2073
Data columns (total 7 columns):
Date_Posted     1075 non-null datetime64[ns]
Neighborhood    1075 non-null object
Post_Title      1075 non-null object
URL             1075 non-null object
Price           1075 non-null int64
Sqft            1075 non-null float64
Bedrooms        1075 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 67.2+ KB


In [71]:
to_rooms.describe()

,Price,Sqft,Bedrooms
count,1075.000000,1075.000000,1075.000000
mean,2342.418605,973.139535,1.720930
std,755.714642,680.308890,0.948526
min,750.000000,110.000000,1.000000
25%,1980.000000,591.000000,1.000000
50%,2300.000000,705.000000,1.000000
75%,2650.000000,1100.000000,2.000000
max,4500.000000,4000.000000,5.000000


In [72]:
to_rooms.head(200)

,Date_Posted,Neighborhood,Post_Title,URL,Price,Sqft,Bedrooms
0,2020-01-11 15:46:00,400 Walmer Rd,very spacious unit parking included mins to vi...,https://toronto.craigslist.org/tor/apa/d/toron...,4200,1300.0,3.0
3,2020-01-11 15:36:00,Toronto Bay/College,Luxurious College Park condo for rent,https://toronto.craigslist.org/tor/apa/d/downt...,2400,640.0,1.0
6,2020-01-11 15:30:00,Toronto,Available now 1 bedroom apt downtown (sherbour...,https://toronto.craigslist.org/tor/apa/d/toron...,1450,550.0,1.0
9,2020-01-11 15:25:00,Etobicoke,"Stylish 2 Bedroom, 1 Bath with Hardwood Floors .",https://toronto.craigslist.org/tor/apa/d/etobi...,1350,900.0,2.0
10,2020-01-11 15:22:00,85 Wood StPh,brand new never lived in before + den mins to ...,https://toronto.craigslist.org/tor/apa/d/toron...,2400,570.0,1.0
...,...,...,...,...,...,...,...
367,2020-01-11 13:13:00,Rogers & Caledonia,"Beautiful, Luxurious Newly Renovated 3 Bedroom...",https://toronto.craigslist.org/tor/apa/d/york-...,3250,1600.0,3.0
370,2020-01-11 12:56:00,Eglinton & Midland,$2295-All inclusive 2Bedroom Condo Townhouse-E...,https://toronto.craigslist.org/tor/apa/d/scarb...,2250,800.0,2.0
374,2020-01-11 12:50:00,Border of Vaughan and Toronto,1 bedroom large den,https://toronto.craigslist.org/yrk/apa/d/thorn...,2199,700.0,1.0
380,2020-01-11 12:29:00,Toronto,Furnished 2-Bedroom Basement Apt at Queen West...,https://toronto.craigslist.org/tor/apa/d/toron...,1980,900.0,2.0


In [73]:
#Before applying the command I have 386 rows
#Lets drop the drop the duplicate with the same title
to_rooms = to_rooms.drop_duplicates(subset='Post_Title') 


In [74]:
print(to_rooms)

           Date_Posted                                       Neighborhood  \
0  2020-01-11 15:46:00                                      400 Walmer Rd   
3  2020-01-11 15:36:00                             Toronto    Bay/College   
6  2020-01-11 15:30:00                                            Toronto   
9  2020-01-11 15:25:00                                          Etobicoke   
10 2020-01-11 15:22:00                                       85 Wood StPh   
11 2020-01-11 15:22:00                                           Brampton   
13 2020-01-11 15:17:00                                19 Grand Trunk Cres   
14 2020-01-11 15:14:00                     10 minutes walk to Subway !!!!   
15 2020-01-11 15:14:00                     City of Toronto Yonge Eglinton   
16 2020-01-11 15:07:00                                  Sheppard/DonMills   
17 2020-01-11 15:01:00                                        Dundas St W   
18 2020-01-11 14:50:00                                            Toronto   

In [75]:
to_rooms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 0 to 81
Data columns (total 7 columns):
Date_Posted     43 non-null datetime64[ns]
Neighborhood    43 non-null object
Post_Title      43 non-null object
URL             43 non-null object
Price           43 non-null int64
Sqft            43 non-null float64
Bedrooms        43 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 2.7+ KB


In [76]:
#Now I have 45 entries only

In [77]:
to_rooms.head(100)

,Date_Posted,Neighborhood,Post_Title,URL,Price,Sqft,Bedrooms
0,2020-01-11 15:46:00,400 Walmer Rd,very spacious unit parking included mins to vi...,https://toronto.craigslist.org/tor/apa/d/toron...,4200,1300.0,3.0
3,2020-01-11 15:36:00,Toronto Bay/College,Luxurious College Park condo for rent,https://toronto.craigslist.org/tor/apa/d/downt...,2400,640.0,1.0
6,2020-01-11 15:30:00,Toronto,Available now 1 bedroom apt downtown (sherbour...,https://toronto.craigslist.org/tor/apa/d/toron...,1450,550.0,1.0
9,2020-01-11 15:25:00,Etobicoke,"Stylish 2 Bedroom, 1 Bath with Hardwood Floors .",https://toronto.craigslist.org/tor/apa/d/etobi...,1350,900.0,2.0
10,2020-01-11 15:22:00,85 Wood StPh,brand new never lived in before + den mins to ...,https://toronto.craigslist.org/tor/apa/d/toron...,2400,570.0,1.0
11,2020-01-11 15:22:00,Brampton,House for rent,https://toronto.craigslist.org/bra/apa/d/bramp...,2500,2500.0,4.0
13,2020-01-11 15:17:00,19 Grand Trunk Cres,1+1 Bedroom Condo Unit In The Heart Of Downtow...,https://toronto.craigslist.org/tor/apa/d/toron...,2500,650.0,1.0
14,2020-01-11 15:14:00,10 minutes walk to Subway !!!!,Bachelor Apartment - walk to Subway !!!!!!,https://toronto.craigslist.org/tor/apa/d/toron...,1350,500.0,1.0
15,2020-01-11 15:14:00,City of Toronto Yonge Eglinton,Heart of Yonge and Eglinton - 2 Bedroom,https://toronto.craigslist.org/tor/apa/d/toron...,2500,1000.0,2.0
16,2020-01-11 15:07:00,Sheppard/DonMills,NORTH YORK 2BEDS 2BATHS CONDO 1PARKING INCLD,https://toronto.craigslist.org/tor/apa/d/north...,2600,850.0,2.0
